In [9]:
import json
import networkx as nx
from dimod import ConstrainedQuadraticModel, Binary, quicksum
from dwave.system import LeapHybridCQMSampler

def build_cqm(G, num_colors):
    """Build CQM model."""


    # Initialize the CQM object
    cqm = ConstrainedQuadraticModel()

    # Build CQM variables
    colors = {n: {c: Binary((n, c)) for c in range(num_colors)} for n in G.nodes}

    # Add constraint to make variables discrete
    for n in G.nodes():
        cqm.add_discrete([(n, c) for c in range(num_colors)])
  
    # Build the constraints: edges have different color end points
    for u, v in G.edges:
        for c in range(num_colors):
            cqm.add_constraint(colors[u][c]*colors[v][c] == 0)

    return cqm

def run_hybrid_solver(cqm):
    """Solve CQM using hybrid solver."""

    # Initialize the CQM solver
    sampler = LeapHybridCQMSampler(token='YOUR_DWAVE_TOKEN')

    # Solve the problem using the CQM solver
    sampleset = sampler.sample_cqm(cqm, label='Example - Graph Coloring')
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    try:
        sample = feasible_sampleset.first.sample
    except Exception as e:
        print(e)
        return False

    soln = {key[0]: key[1] for key, val in sample.items() if val == 1.0}

    return soln

In [10]:
data={"data": {"edges": [[0, 1], [0, 2], [1, 3], [2, 3], [1, 4],[3,4]]}}

In [11]:
import networkx as nx
G=nx.Graph()
for x in data['data']['edges']:
    G.add_edge(x[0],x[1])
num_colors = max(d for _, d in G.degree()) +1
res=None
follow=True
while(follow==True):
    cqm = build_cqm(G, num_colors)
    sample = run_hybrid_solver(cqm)
    if(sample is not False):
        res=sample
        num_colors=num_colors-1
    else:
        follow=False
r=[[] for i in range(max([res[x] for x in res])+1)]
for x in res:
    r[res[x]].append(x)
output={}
output['groups']={"colour_"+str(i+1):r[i] for i in range(len(r))}
output['number_of_colours']=len(r)
output

SampleSet is empty


{'groups': {'colour_1': [1, 2], 'colour_2': [0, 3], 'colour_3': [4]},
 'number_of_colours': 3}